In [6]:
!pip3 install catboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 6.6 MB/s eta 0:00:000m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 2.6 MB/s eta 0:00:00


In [1]:
!pip3 install xgboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [2]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [3]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [201]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [160]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [4]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# get average age in train/test
avg_age = (train_df["Age"].mean() + test_df["Age"].mean()) / 2
avg_age

29.985854004252303

In [6]:
# Fill age with mean
train_df[['Age']] = train_df[['Age']].fillna(value=avg_age)
test_df[['Age']] = test_df[['Age']].fillna(value=avg_age)

In [7]:
# Fill cabin to -999
train_df.fillna(-999,inplace=True)
test_df.fillna(-999,inplace=True)

In [8]:
x = train_df.drop('Survived',axis=1) # train df without survived status
y = train_df.Survived # df with passenger id and survived status

In [9]:
x.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [87]:
# Convert floats to ints, because CatBoost can't use floats
train_df = train_df.astype({"Age": int, "Fare": int})
test_df = test_df.astype({"Age": int, "Fare": int})

In [88]:
import numpy as np
# We will use all indexes after convert
cate_features_index = np.where(x.dtypes != float)[0]

In [89]:
cate_features_index

array([ 0,  1,  2,  3,  5,  6,  7,  9, 10])

In [90]:
from sklearn.model_selection import train_test_split

# Split data to train and valid
xtrain, xtest, ytrain, ytest = train_test_split(x,y,train_size=.85,random_state=1234)

In [91]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42, loss_function='Logloss', verbose=False)

In [92]:
model.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest, ytest))

In [93]:
from sklearn.metrics import accuracy_score

print('the test accuracy is :{:.6f}'.format(accuracy_score(ytest,model.predict(xtest))))

the test accuracy is :0.850746


In [94]:
params = model.get_params()
#params["loss_funcion"] = "Logloss"
params

{'loss_function': 'Logloss',
 'random_seed': 42,
 'use_best_model': True,
 'verbose': False,
 'eval_metric': 'Accuracy'}

In [113]:
# Cross val
from catboost import Pool, cv

cv_data = cv(Pool(x,y,cat_features=cate_features_index), params,fold_count=10)


In [107]:
print('the best cv accuracy is :{}'.format(np.max(cv_data["test-Accuracy-mean"])))

the best cv accuracy is :0.8406838043354897


In [108]:
cv_data

,iterations,test-Accuracy-mean,test-Accuracy-std,train-Accuracy-mean,train-Accuracy-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.780094,0.037240,0.797730,0.009794,0.676262,0.001849,0.676301,0.001309
1,1,0.783465,0.038267,0.799101,0.009375,0.661509,0.003435,0.661459,0.001826
2,2,0.790170,0.028576,0.801467,0.010924,0.647764,0.004870,0.647514,0.002798
3,3,0.791243,0.023796,0.801095,0.011798,0.634806,0.005769,0.634275,0.003231
4,4,0.790119,0.030452,0.803216,0.011565,0.622134,0.007363,0.621504,0.003484
...,...,...,...,...,...,...,...,...,...
995,995,0.832768,0.043511,0.898866,0.005374,0.395839,0.075936,0.289186,0.006270
996,996,0.833891,0.042313,0.898991,0.005226,0.395924,0.076072,0.289103,0.006294
997,997,0.833891,0.042313,0.898866,0.005374,0.395903,0.076022,0.289011,0.006287
998,998,0.833891,0.042313,0.898991,0.005516,0.395928,0.075978,0.288951,0.006288


In [109]:
pred = model.predict(test_df)
pred = pred.astype(np.int)
submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':pred})
submission.to_csv('catboost_solution.csv',index=False)


/tmp/ipykernel_1396006/296897289.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred = pred.astype(np.int)


In [112]:
#lightgbm boosting
!pip3 install lightgbm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [121]:
xtrain.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [18]:
xtrain["Sex"] = xtrain["Sex"].astype('category').cat.codes

In [19]:
xtrain["Embarked"] = xtrain["Embarked"].astype('category').cat.codes

In [20]:
xtrain["Ticket"] = xtrain["Ticket"].astype('category').cat.codes

In [21]:
# Delete names
xtrain = xtrain.drop('Name',axis=1) # train df without survived status

In [22]:
xtrain["Cabin"] = xtrain["Cabin"].astype('category').cat.codes

In [104]:
xtrain

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
848,849,2,"Harper, Rev. John",male,28.000000,0,1,248727,33.0000,-999,S
855,856,3,"Aks, Mrs. Sam (Leah Rosen)",female,18.000000,0,1,392091,9.3500,-999,S
746,747,3,"Abbott, Mr. Rossmore Edward",male,16.000000,1,1,C.A. 2673,20.2500,-999,S
355,356,3,"Vanden Steen, Mr. Leo Peter",male,28.000000,0,0,345783,9.5000,-999,S
61,62,1,"Icard, Miss. Amelie",female,38.000000,0,0,113572,80.0000,B28,-999
...,...,...,...,...,...,...,...,...,...,...,...
204,205,3,"Cohen, Mr. Gurshon ""Gus""",male,18.000000,0,0,A/5 3540,8.0500,-999,S
53,54,2,"Faunthorpe, Mrs. Lizzie (Elizabeth Anne Wilkin...",female,29.000000,1,0,2926,26.0000,-999,S
294,295,3,"Mineff, Mr. Ivan",male,24.000000,0,0,349233,7.8958,-999,S
723,724,2,"Hodges, Mr. Henry Price",male,50.000000,0,0,250643,13.0000,-999,S


In [103]:
xtest.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [25]:
xtest["Sex"] = xtest["Sex"].astype('category').cat.codes
xtest["Embarked"] = xtest["Embarked"].astype('category').cat.codes
xtest["Ticket"] = xtest["Ticket"].astype('category').cat.codes
xtest["Cabin"] = xtest["Cabin"].astype('category').cat.codes
xtest = xtest.drop('Name',axis=1) # train df without survived status

KeyError: "['Name'] not found in axis"

In [101]:
import lightgbm as lgb


lgbm_model = lgb.LGBMClassifier(num_iterations=650, learning_rate=0.005, max_bin=200000)

lgbm_model.fit(xtrain, ytrain)


/home/stephan/.local/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: Name, Sex, Ticket, Cabin, Embarked

In [81]:
ypred = lgbm_model.predict(xtest)

In [82]:
ypred

array([1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0])

In [83]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(ypred, ytest)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(ytest, ypred)))


LightGBM Model accuracy score: 0.8507


In [85]:
print('Training set score: {:.4f}'.format(lgbm_model.score(xtrain, ytrain)))

print('Test set score: {:.4f}'.format(lgbm_model.score(xtest, ytest)))


Training set score: 0.9234
Test set score: 0.8507


In [439]:
test_df["Sex"] = test_df["Sex"].astype('category').cat.codes
test_df["Embarked"] = test_df["Embarked"].astype('category').cat.codes
test_df["Ticket"] = test_df["Ticket"].astype('category').cat.codes
test_df["Cabin"] = test_df["Cabin"].astype('category').cat.codes
test_df = test_df.drop('Name',axis=1) # train df without survived status
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,1,34,0,0,152,7,0,1
1,893,3,0,47,1,0,221,7,0,2
2,894,2,1,62,0,0,73,9,0,1
3,895,3,1,27,0,0,147,8,0,2
4,896,3,0,22,1,1,138,12,0,2
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,29,0,0,267,8,0,2
414,1306,1,0,39,0,0,324,108,23,0
415,1307,3,1,38,0,0,346,7,0,2
416,1308,3,1,29,0,0,220,8,0,2


In [440]:
pred = clf.predict(test_df)
pred = pred.astype(np.int)
submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':pred})
submission.to_csv('lightgbm.csv',index=False)


/tmp/ipykernel_1396006/1613228742.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred = pred.astype(np.int)


In [75]:
# XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', learning_rate=0.008, max_bin=200000, max_leaves=100)


/home/stephan/.local/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [76]:
xgb_model.fit(xtrain, ytrain)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.008, max_bin=200000, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=100, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [77]:
ypred = xgb_model.predict(xtest)

In [78]:
ypred

array([1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1])

In [79]:
accuracy = accuracy_score(ytest, ypred)
print("Test acc:", accuracy)
print("Train acc:", xgb_model.score(xtrain, ytrain))

Test acc: 0.8507462686567164
Train acc: 0.9035667107001321


In [95]:
catboost_model = model

In [105]:
cb_pred = catboost_model.predict(test_df)
cb_pred = cb_pred.astype(np.int)

lgbm_pred = xgb_model.predict(test_df)
lgbm_pred = lgbm_pred.astype(np.int)
print(cb_pred)
print(lgbm_pred)

/tmp/ipykernel_2455926/1305463340.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cb_pred = cb_pred.astype(np.int)


ValueError: Feature shape mismatch, expected: 10, got 11